In [2]:
import pandas as pd
import numpy as np
import json
import mysql.connector
import datetime
import pytz

In [3]:
with open('new_Isin_to_masterid.json', 'r') as f:
    isinToMasterid = json.load(f)
with open('Column_to_indicator_map.json', 'r', encoding="utf-8") as f:
    colToIndicator = json.load(f)

In [4]:
binaryColumns = [
    'amortizing security (yes/no)',
    "floating rate (yes/no)",
    "mortgage bonds (yes/no)",
    "perpetual (yes/no)",
    "restructing (yes/no)",
    "covered debt (yes/no)",
    "structured products (yes/no)",
    "subordinated debt (yes/no)",
    "dual currency bond",
    "foreign bond",
    "the flag that paper green bonds",
    "pik",
    "non-market issue (yes/no)",
    "redemption linked (yes/no)",
    "retail bonds",
    "securitisation",
    "sukuk (yes/no)",
    "trace-eligible securities"
]

In [5]:
dateColumns = [
    "end of placement date",
    "start of trading",
    "start of placement date",
    "early redemption date",
    "maturity date",
    "next offer date (put/call) - to be specified",
    "settlement date",
    "current coupon date",
    "next offer date (call)",
    "next offer date (put)",
]

In [6]:
ts_date = '2021-12-06'
# today_date = datetime.datetime.today().strftime('%Y-%m-%d')
today_date = '2022-01-10'
path = r'D:\\sriram\\agrud\\cbonds_data entry\\test_files\\zip_files\\'+today_date+'\\'
# publish_date = (datetime.datetime.strptime(publish_date, '%Y-%m-%d').date() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
try:
    df = pd.read_excel(path+'issues.xls')
except:
    df = pd.read_csv(path+'issues.csv', sep=';', encoding='latin')
df.columns = df.columns.str.strip().str.lower()

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\\\sriram\\\\agrud\\\\cbonds_data entry\\\\test_files\\\\zip_files\\\\2022-01-10\\issues.csv'

In [5]:
df['master id'] = df['isin / isin regs'].map(isinToMasterid)
df[binaryColumns] = df[binaryColumns].replace([[0], [1]], [["No"], ["Yes"]])
if "сonvertable (yes/no)" in df.columns :
    df['сonvertable (yes/no)'] = df['сonvertable (yes/no)'].map({0 : "No", 1 : "Yes"})
if "ñonvertable (yes/no)" in df.columns :
    df["ñonvertable (yes/no)"] = df["ñonvertable (yes/no)"].map({0 : "No", 1 : "Yes"})

datetimeColumns = pd.to_datetime(df[dateColumns].stack(), format="%d.%m.%Y").unstack()
df[datetimeColumns.columns] = datetimeColumns
for col in df.select_dtypes('datetime').columns:
    df[col] = df[col]

df.dropna(subset=['master id'],inplace=True)
df = df.replace([np.NaN], ['NA'])
df = df.replace([pd.NaT], ['NA'])

In [6]:
result = []
for i, row in df.iterrows():
    row2 = row.to_dict()
    masterId = row2['master id']
    for k, v in row2.items():
        if v == "NA":
            continue
        if k in colToIndicator:
            try:
                indicatorId = colToIndicator[k]
                if type(v) == pd.Timestamp:
                    json_data = None
                    dataType = 2
                    date = datetime.datetime.strftime(v,'%Y-%m-%d')
                    gmt = pytz.timezone("GMT")
                    value_data = gmt.localize(datetime.datetime.strptime(date, '%Y-%m-%d')).timestamp()
                elif type(v) == float or type(v) == int or  v.isnumeric(): 
                    dataType = 0
                    value_data = v
                    json_data = None
                elif v.isnumeric() == False:
                    json_data = json.dumps({'TEXT':v})
                    dataType = 3
                    value_data = 0
            except Exception as e:
                print(k)
                print(v)
                print(e)
            result.append([masterId,indicatorId,value_data,json_data,dataType,ts_date])

In [8]:
try:
    db_conn = mysql.connector.connect(host='54.237.79.6',database='rentech_db',user='rentech_user',password='N)baegbgqeiheqfi3e9314jnEkekjb',auth_plugin='mysql_native_password')
    cursor = db_conn.cursor()
    sql = """INSERT INTO `raw_data` (`id`, `master_id`, `indicator_id`, `value_data`, `json_data`, `data_type`, `ts_date`, `ts_hour`, `job_id`, `timestamp`) VALUES 
    (NULL, %s, %s, %s, %s, %s, %s, '0:0:0', 9, NOW())
    ON DUPLICATE KEY UPDATE master_id = VALUES(master_id), indicator_id = VALUES(indicator_id), value_data = VALUES(value_data),
    json_data = VALUES(json_data),data_type = VALUES(data_type), ts_date = VALUES(ts_date) ,ts_hour = VALUES(ts_hour) ,
    job_id = VALUES(job_id), batch_id = VALUES(batch_id);"""
    cursor.executemany(sql, result)
    db_conn.commit()
    print(f'{cursor.rowcount} rows inserted successfully')
except Exception as e:
        print ("Error while connecting to MySQL using Connection pool ", e)
finally:
    if(db_conn.is_connected()):
        cursor.close()
        db_conn.close()
        print("MySQL connection is closed")

0 rows inserted successfully
MySQL connection is closed
